In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

import gensim
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
import nltk

from gensim.matutils import Sparse2Corpus

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import LdaMulticore, CoherenceModel

from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim
import time

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import cluster, metrics

from sklearn import manifold, decomposition





In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### Chargement de données

Les données ont été récupéré du site StackOverFlow en intégrant des filtres à la requette SQL (filtres sur le nombre de tags par question, le score ... )

In [ ]:
df = pd.read_csv('data\dataframe.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

# 1 - APPROCHE NON SUPERVISEE

Partition des données en données d'entrainement et données test\


In [ ]:
# Créer une nouvelle colonne qui concatène 'title' et 'body'
df['text'] = df['Title'] + ' ' + df['Body']

# Diviser le dataframe en train et test avec un ratio de 0.2
X_train, X_test = train_test_split(df,  test_size=0.2)

Réalisation de bag of word en utilisant CountVectorizer

In [ ]:
# Créer un objet CountVectorizer pour vectoriser les textes
vectorizer = CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1,2))

# Transformer les textes en une matrice de comptage de mots
vect_train = vectorizer.fit_transform(X_train['text'])
vect_test = vectorizer.transform(X_test['text'])

Le modèle LDA permet d'extraire des topics (ou thèmes) à partir d'un corpus de textes en utilisant un apprentissage non supervisé.

In [ ]:
#Créer un objet Dictionary à partir des mots du vectorizer
dictionary = Dictionary.from_corpus(Sparse2Corpus(vect_train, documents_columns=False), id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))

#Créer un objet Corpus à partir de la matrice vect_train
corpus = Sparse2Corpus(vect_train, documents_columns=False)
#Transformer le corpus en une liste de listes de mots en utilisant le dictionnaire
texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]

perplexity

In [ ]:
#Définir l'interval de nombres de topics à tester
num_topics_range = range (2, 10)

#Créer une liste vide pour stocker les valeur du score 'perplexity'
perplexities = []

#Boucler sur les nombres de topics
for num_topics in num_topics_range:
    #Créer un modèle LDA avec le nombre de topics courant
    lda_per = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)
    #Calculer la perplexité du modèle sur le corpus de test
    perplexity = lda_per.log_perplexity(Sparse2Corpus(vect_test, documents_columns=False))
    #Ajouter la perplexité à la liste
    perplexities.append(perplexity)
    print(f"perplexity avec {num_topics} topics: {perplexity}")


In [ ]:
#Tracer un graphique avec les nombres de topics en abscisse et les perplexités en ordonnée
plt.figure()
plt.plot(num_topics_range, perplexities)
plt.xlabel('Nombre topics')
plt.ylabel('Perplexity')
plt.title('Perplexity du model en fonction de différents nombres de topics')
plt.show()

In [ ]:
best_index = np.argmin(perplexities)

#Trouver le nombre de topics correspondant
best_num_topics = num_topics_range[best_index]

#Afficher le meilleur nombre de topics
print(f"Le meilleur nombre de topics pour le modèle est {best_num_topics}")


Coherence score

Le coherence score est une mesure qui évalue la cohérence des mots-clés d'un topic en fonction de leur co-occurrence dans les documents. Plus le coherence score est élevé, plus le topic est cohérent et interprétable.

Pour calculer le coherence score, il faut utiliser la classe CoherenceModel de gensim et lui passer le modèle LDA, le corpus, le dictionnaire et la méthode de cohérence à utiliser. Il existe plusieurs méthodes de cohérence, mais la plus courante est la méthode 'c_v'.

Pour déterminer le nombre de topics optimum, il faut tester plusieurs valeurs de num_topics et comparer les coherence scores correspondants. On va utiliser une boucle for pour créer plusieurs modèles LDA avec différentes valeurs de num_topics et calculer leur coherence score avec la méthode get_coherence() de CoherenceModel. On peut ensuite tracer un graphique qui montre l'évolution du coherence score en fonction du nombre de topics et choisir la valeur qui maximise le coherence score.

In [ ]:
num_topics_range = range(3, 17)

#réer une liste vide pour stocker les coherence scores
coherence_scores = []

#Créer une boucle for pour tester chaque valeur de num_topics
for num_topics in num_topics_range:

    #Créer un objet LDA avec la valeur courante de num_topics
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

    #Créer un objet CoherenceModel avec la méthode 'c_v'
    cm = CoherenceModel(model=lda, corpus=corpus, dictionary=dictionary, coherence='c_v', texts=texts)


    #Calculer le coherence score et l'ajouter à la liste
    coherence_score = cm.get_coherence()
    coherence_scores.append(coherence_score)

    #Afficher le coherence score pour la valeur courante de num_topics
    print(f"Coherence score avec {num_topics} topics: {coherence_score}")


In [ ]:

#Tracer un graphique qui montre l'évolution du coherence score en fonction du nombre de topics
plt.plot(num_topics_range, coherence_scores)
plt.xlabel('Nombre topics')
plt.ylabel('Coherence score')
plt.title('Coherence score en fonction du nombre de topics')
plt.show()

#Choisir le nombre de topics optimum qui maximise le coherence score
num_topics_opt = num_topics_range[np.argmax(coherence_scores)]
print(f"Le nombre de topic optimum est {num_topics_opt}")

In [ ]:
#Créer un objet LDA avec le nombre de topics optimum
lda_opt = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics_opt, random_state=42)

#Afficher les mots-clés de chaque topic
words = vectorizer.get_feature_names()
for i, topic in enumerate(lda_opt.show_topics(formatted=False)):
    print(f"Topic {i+1}:")
    print(" ".join([word for word, weight in topic[1]]))
    print("\n")

#Afficher la distribution des topics pour chaque question
topics = lda_opt.get_document_topics((corpus), minimum_probability=0)
for i, question in enumerate(X_train['text']):
    print(f"Question {i+1}: {question}")
    print(f"Topic distribution: {[prob for topic, prob in topics[i]]}")
    print("\n")

In [ ]:
data_opt = pyLDAvis.gensim_models.prepare(lda_opt, corpus, dictionary)

pyLDAvis.display(data_opt)

In [ ]:
num_topics_opt = 6

In [ ]:
#Créer un objet LDA avec le nombre de topics optimum
lda_opt = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics_opt, random_state=42)

#Afficher les mots-clés de chaque topic
words = vectorizer.get_feature_names()
for i, topic in enumerate(lda_opt.show_topics(formatted=False)):
    print(f"Topic {i+1}:")
    print(" ".join([word for word, weight in topic[1]]))
    print("\n")

#Afficher la distribution des topics pour chaque question
topics = lda_opt.get_document_topics((corpus), minimum_probability=0)
for i, question in enumerate(X_train['text']):
    print(f"Question {i+1}: {question}")
    print(f"Topic distribution: {[prob for topic, prob in topics[i]]}")
    print("\n")

In [ ]:
data_opt = pyLDAvis.gensim_models.prepare(lda_opt, corpus, dictionary)

pyLDAvis.display(data_opt)

In [ ]:
num_topics_opt = 5

In [ ]:
#Créer un objet LDA avec 4 topics 
lda_opt = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics_opt, random_state=42)

#Afficher les mots-clés de chaque topic
words = vectorizer.get_feature_names()
for i, topic in enumerate(lda_opt.show_topics(formatted=False)):
    print(f"Topic {i+1}:")
    print(" ".join([word for word, weight in topic[1]]))
    print("\n")

#Afficher la distribution des topics pour chaque question
topics = lda_opt.get_document_topics((corpus), minimum_probability=0)
for i, question in enumerate(X_train['text']):
    print(f"Question {i+1}: {question}")
    print(f"Topic distribution: {[prob for topic, prob in topics[i]]}")
    print("\n")

Analyse des topics et des mots associés (pyLDAVIS)

In [ ]:
data_opt = pyLDAvis.gensim_models.prepare(lda_opt, corpus, dictionary)

pyLDAvis.display(data_opt)

Mtopics-words est une matrice qui contient les poids des mots pour chaque topic. Chaque ligne correspond à un topic et chaque colonne correspond à un mot. La valeur à la position (i, j) indique le poids du mot j dans le topic i. Le poids peut être interprété comme la probabilité du mot dans le topic ou comme le score de pertinence du mot pour le topic selon la méthode utilisée

In [ ]:
#Obtenir la matrice Mtopics-words à partir du modèle LDA
Mtopics_words = lda_opt.get_topics()
#Mtopics_words = np.where(Mtopics_words < 0.05, 0, Mtopics_words)


#Créer un dataframe à partir de la matrice Mtopics-words
df = pd.DataFrame(Mtopics_words, index=['Topic '+str(i+1) for i in range(num_topics_opt)], columns=vectorizer.get_feature_names())

"""#Tracer la carte de chaleur
plt.figure(figsize=(15,10))
sns.heatmap(df, cmap='Oranges')
plt.title('Mtopics-words matrix')
plt.show()"""


#Sélectionner 200 colonnes aléatoires du dataframe df
df_sample = df.sample(n=50, axis=1)

#Tracer la carte de chaleur avec le dataframe échantillonné
plt.figure(figsize=(15,10))
sns.heatmap(df_sample, cmap='Oranges')
plt.title('Mtopics-words matrix')
plt.show()

In [ ]:
df

In [ ]:
Mtopics_words

In [ ]:
'''
#Obtenir la matrice Mtopics-words à partir du modèle LDA
Mtopics_words = lda_opt.get_topics()

#Obtenir les noms des mots du vocabulaire
words = vectorizer.get_feature_names()

#Obtenir le nombre de sujets et de mots
num_topics = Mtopics_words.shape[0]
num_words = Mtopics_words.shape[1]

#Créer une figure et un axe
fig, ax = plt.subplots(figsize=(15,10))

#Créer un tableau pour stocker les positions des barres
bottom = np.zeros(num_words)

#Créer une liste de couleurs
colors = plt.cm.Oranges(np.linspace(0.2, 1, num_topics))

#Boucler sur les sujets
for i in range(num_topics):
    #Tracer une barre pour le sujet i avec la couleur i
    ax.bar(words, Mtopics_words[i], bottom=bottom, color=colors[i], label='Topic '+str(i+1))
    #Mettre à jour les positions des barres
    bottom += Mtopics_words[i]

#Ajouter un titre et des légendes
ax.set_title('Mtopics-words matrix')
ax.set_ylabel('Probability')
ax.legend(loc='upper right')

#Afficher le diagramme
plt.show()
'''

Les valeurs de la matrice M(train)quest-topics représentent les probabilités pour chaque question d'appartenir à chacun des topics. Plus la valeur est proche de 1, plus la question est fortement liée au topic correspondant. Plus la valeur est proche de 0, plus la question est faiblement liée au topic correspondant.

In [ ]:
#Obtenir la liste des distributions de topics pour chaque question du train
topics_train = lda_opt.get_document_topics(corpus)

#Obtenir le nombre de questions et de topics
num_questions = len(topics_train)
num_topics = lda_opt.num_topics

#Créer une matrice numpy de zéros avec la forme (num_questions, num_topics)
Mtrain_quest_topics = np.zeros((num_questions, num_topics))

#Boucler sur les questions
for i in range(num_questions):
    #Obtenir la distribution de topics pour la question i
    topics = topics_train[i]
#Boucler sur les tuples (topic_id, probability)
for topic_id, probability in topics:
    #Remplir la matrice avec la probabilité à l'indice (i, topic_id)
    np.put(Mtrain_quest_topics, [i*num_topics + topic_id], probability)

#Mtrain_quest_topics = np.where(Mtrain_quest_topics < 0.01, 0, Mtrain_quest_topics)

#Afficher la matrice
print(Mtrain_quest_topics)

In [ ]:
#Obtenir le topic dominant pour chaque question
topics_dominant = np.argmax(Mtrain_quest_topics, axis=1)

#Afficher les topics dominants
print(topics_dominant)

In [ ]:
lda_opt.show_topic(0)


In [ ]:
lda_opt.show_topic(1)


In [ ]:
Mtrain_quest_topics.shape

In [ ]:
Mtopics_words.shape

In [ ]:
Mtrain_quest_topics

In [ ]:
Mtopics_words

Probabilité des mots pour les questions train

In [ ]:
#Multiplier les deux matrices M(train)quest-topics et Mtopics-words
#Mtrain_quest_words = np.dot(Mtrain_quest_topics, Mtopics_words)

#Afficher la matrice M(train)quest-words
#print(Mtrain_quest_words)


#Multiplier les deux matrices M(train)quest-topics et Mtopics-words
#Mtrain_quest_words = np.dot(Mtrain_quest_topics[:1000, :], Mtopics_words[:, :100])
Mtrain_quest_words = np.matmul(Mtrain_quest_topics[:1000, :], Mtopics_words[:, :100])


#Afficher la matrice M(train)quest-words
print(Mtrain_quest_words)

Probabilité de mots pour les questions test

In [ ]:
#Convertir la matrice vect_test en un objet Corpus
corpus_test = Sparse2Corpus(vect_test, documents_columns=False)

#Obtenir la distribution de topics pour chaque question du test
topics_test = lda_opt.get_document_topics(corpus_test)

P(mot | question) = P(mot | topic) * P(topic | question)


In [ ]:
#Obtenir la matrice M(test)quest-topics
Mtest_quest_topics = np.zeros((len(topics_test), lda_opt.num_topics))
for i in range(len(topics_test)):
    topics = topics_test[i]
    
for topic_id, probability in topics:
    np.put(Mtest_quest_topics, [i*lda_opt.num_topics + topic_id], probability)

#Obtenir la matrice Mtopics-words
Mtopics_words = lda_opt.get_topics()

#Multiplier les deux matrices M(test)quest-topics et Mtopics-words
Mtest_quest_words = np.dot(Mtest_quest_topics[:1000, :], Mtopics_words[:, :100])


#Afficher la matrice M(test)quest-words
print(Mtest_quest_words)

In [ ]:
i = 0

#Afficher la question et la distribution de topics
print(f"Question {i+1}: {X_test['text'].iloc[i]}")
print(f"Topic distribution: {[prob for topic, prob in topics_test[i]]}")

Ce code permettra d'obtenir une liste de valeurs de cohérence pour chaque seuil testé.

In [ ]:
"""
# Créer une liste vide pour stocker les valeurs de cohérence
coherence_values = []

# Faire une boucle for qui teste les seuils entre 0 et 1, avec un pas de 0.01
for threshold in np.arange(0, 1, 0.01):

    # Appliquer la fonction np.where aux matrices avec le seuil courant
    Mtrain_quest_topics_threshold = np.where(Mtrain_quest_topics < threshold, 0, Mtrain_quest_topics)
    Mtopics_words_threshold = np.where(Mtopics_words < threshold, 0, Mtopics_words)

    # Créer un modèle de cohérence avec le modèle LDA et les matrices seuillées
    #coherence_model_lda = CoherenceModel(model=lda_opt, topics=Mtopics_words_threshold, texts=Mtrain_quest_topics_threshold, coherence='c_v')
    coherence_model_lda = CoherenceModel(model=lda_opt, dictionary=dictionary, topics=Mtopics_words_threshold, texts=Mtrain_quest_topics_threshold, coherence='c_v')

    # Calculer la cohérence du modèle et l'ajouter à la liste
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)
    print(f"Coherence score avec {threshold} threshold: {coherence_score}")
"""




Ce code permettra d'afficher les topics et les mots pour chaque seuil testé.

In [ ]:
"""
# Créer une liste vide pour stocker les topics et les mots
topics_words = []

# Faire une boucle for qui teste les seuils entre 0 et 1, avec un pas de 0.01
for threshold in np.arange(0, 1, 0.01):

# Appliquer la fonction np.where aux matrices avec le seuil courant
Mtrain_quest_topics_threshold = np.where(Mtrain_quest_topics < threshold, 0, Mtrain_quest_topics)
Mtopics_words_threshold = np.where(Mtopics_words < threshold, 0, Mtopics_words)

# Créer un nouveau modèle LDA avec les matrices seuillées
lda_model_threshold = LdaModel(Mtrain_quest_topics_threshold, num_topics=10, id2word=Mtopics_words_threshold)

# Afficher les topics et les mots avec la fonction show_topic()
print(f"Seuil: {threshold}")
for topic in range(10):
print(f"Topic {topic}: {lda_model_threshold.show_topic(topic)}")

# Ajouter les topics et les mots à la liste
topics_words.append(lda_model_threshold.show_topics())
"""

Ce code permet d'obtenir une matrice de similarité cosinus entre les questions train et test. Chaque élément de la matrice représente la similarité cosinus entre une question train et une question test. La similarité cosinus varie entre -1 et 1, où 1 signifie une similarité maximale, 0 signifie une similarité nulle, et -1 signifie une similarité minimale.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité cosinus entre les matrices
similarity_matrix = cosine_similarity(Mtrain_quest_topics, Mtest_quest_topics)

# Afficher la matrice de similarité
print(similarity_matrix)


In [ ]:
# Tracer un histogramme de la matrice de similarité
plt.hist(similarity_matrix.flatten(), bins=20)
plt.xlabel("Similarité cosinus")
plt.ylabel("Fréquence")
plt.title("La similarité cosinus entre les questions train et test")
plt.show()

In [ ]:
# Tracer une carte de chaleur de la matrice de similarité
plt.imshow(similarity_matrix, cmap="hot")
plt.colorbar()
plt.xlabel("Questions test")
plt.ylabel("Questions train")
plt.title("Carte de chaleur de la similarité cosinus entre les questions train et test")
plt.show()

In [ ]:
# Créer le vectorizer TF-IDF 
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000)

# Transformer les données textuelles en vecteurs TF-IDF
vect_train = vectorizer.fit_transform(X_train['text'])

# Convertir la matrice TF-IDF en format compatible avec gensim
corpus = gensim.matutils.Sparse2Corpus(vect_train, documents_columns=False)

# Créer le dictionnaire des mots et de leurs indices 
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

# Créer le modèle LDA avec gensim
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)


In [ ]:
id2word

# 2 - APPROCHE SUPPERVISEE

In [ ]:
# Importation des modules 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [ ]:
df = pd.read_csv('data\dataframe.csv')

In [ ]:
df = df.dropna()

Définition de la target et des features \ Partition des données en train  test

In [ ]:
# Créer une nouvelle colonne qui concatène 'title' et 'body'
df['text'] = df['Title'] + ' ' + df['Body']


In [ ]:
import ast
df["Tags"] = df["Tags"].apply(ast.literal_eval)
print(df["Tags"])

MultiLabelBinarizer est une classe de sklearn.preprocessing qui permet de transformer des labels multiclasse en une matrice binaire indiquant la présence ou l'absence de chaque classe

Partition des données en train et test sets

In [ ]:
# Séparer les données en train et test sets 
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["Tags"], test_size=0.2, random_state=42)

In [ ]:
# Créer une liste des 50 tags les plus fréquents
top_tags = df["Tags"].explode().value_counts().head(50).index.tolist()

In [ ]:
"""
#Suppression des répétitions
top_tags = []
for tg in top_tags0:
    top_tags.append([word for word in tg if word in top_tags0])
top_tags = np.asarray(top_tags)"""

In [ ]:
print(top_tags)

In [ ]:
# Créer un objet MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=top_tags)

# Transformer les tags du train set en une matrice binaire
y_train_bin = mlb.fit_transform(y_train)

# Transformer les tags du test set en une matrice binaire
y_test_bin = mlb.transform(y_test)

# Créer un objet TfidfVectorizer pour transformer les textes en vecteurs de fréquence
tfidf = TfidfVectorizer()

# Transformer les textes du train set en une matrice de fréquence
X_train_tfidf = tfidf.fit_transform(X_train)

# Transformer les textes du test set en une matrice de fréquence
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
tfidf.get_feature_names_out()

In [ ]:
"""tfidf.get_feature_names_out()
df_bow_sklearn = pd.DataFrame(X_train_tfidf.toarray(),columns=vectorizer.get_feature_names())
df_bow_sklearn.head()"""


In [ ]:
print(mlb.classes_)

définition des fonctions communes

In [ ]:
# Prédiction des tags/ approche non suppervisés
def predict_supervised_tags(n,y_test,y_pred):
    for i in range(n):
        print("Prédit : ", mlb.inverse_transform(y_pred)[i])
        print("Valeur réelle : ", mlb.inverse_transform(y_test)[i])
        print()

#### Prédiction des tags en entrainant le modèle LogisticRegression

OneVsRestClassifier est une classe de scikit-learn qui permet de gérer le cas où la cible est multilabel, c'est-à-dire qu'un document peut avoir plusieurs étiquettes. OneVsRestClassifier utilise un estimateur binaire (par exemple LogisticRegression) pour entraîner un classificateur par étiquette, et combine les résultats pour prédire les étiquettes d'un document. 

In [ ]:
# Créer un objet OneVsRestClassifier avec LogisticRegression 
ovr_logreg = OneVsRestClassifier(LogisticRegression())

# Entraîner les classificateurs sur le train set
ovr_logreg.fit(X_train_tfidf, y_train_bin)

# Prédire les tags sur le test set
y_pred_logreg = ovr_logreg.predict(X_test_tfidf)

In [ ]:
predict_supervised_tags(10, y_test_bin, y_pred_logreg)

#### Prédiction des tags en entrainant le modèle SGDClassifier

In [ ]:
ovr_sgd = OneVsRestClassifier(SGDClassifier())

ovr_sgd.fit(X_train_tfidf, y_train_bin)

y_pred_sgd = ovr_sgd.predict(X_test_tfidf)

In [ ]:
predict_supervised_tags(10, y_test_bin, y_pred_sgd)

In [ ]:
import pickle

# Enregistrer le modèle dans un fichier
filename = "ovr_sgd.pickle"
pickle.dump(ovr_sgd, open(filename, "wb"))

# Charger le modèle à partir du fichier
loaded_model = pickle.load(open(filename, "rb"))

# Faire des prédictions avec le modèle chargé
y_pred_sgd = loaded_model.predict(X_test_tfidf)

Le jaccard_score est une mesure de la similarité entre deux ensembles d'étiquettes. Il est défini comme la taille de l'intersection divisée par la taille de l'union de deux ensembles d'étiquetteshttps://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html. Il peut être utilisé pour comparer l'ensemble des étiquettes prédites pour un échantillon à l'ensemble des étiquettes réelles. Plus le jaccard_score est proche de 1, plus les deux ensembles sont similaires. Plus il est proche de 0, plus ils sont différentshttps://www.statology.org/jaccard-similarity/

In [ ]:
from sklearn.metrics import jaccard_score

# Calculer du jaccard_score 
jac_score_logreg = jaccard_score(y_test_bin, y_pred_logreg, average='micro')
jac_score_sgd = jaccard_score(y_test_bin, y_pred_sgd, average='micro')

# Calcul du precision score
pres_score_logreg = precision_score(y_test_bin,y_pred_logreg, average = "macro")
pres_score_sgd = precision_score(y_test_bin,y_pred_sgd, average = "macro")

# Calcul de l'accuracy score
accuracy_logreg = accuracy_score(y_test_bin, y_pred_logreg)
accuracy_sgd = accuracy_score(y_test_bin, y_pred_sgd)

# Comparer les scores
print ("Jaccard score : ")
print("Jaccard score pour LogisticRegression:", jac_score_logreg)
print("Jaccard score pour SGDClassifier:", jac_score_sgd)
print ("\n")

print ("Precision score : ")
print("Precision score pour LogisticRegression:", pres_score_logreg )
print("Precision score pour SGDClassifier:", pres_score_sgd )
print ("\n")

print ("Accuracy score : ")
print(f"Accuracy LogisticRegression: {accuracy_logreg:.2f}")
print(f"Accuracy SGDClassifier: {accuracy_sgd:.2f}")



## 3 -  Sentence Embedding

In [ ]:
df = pd.read_csv('data\dataframe.csv')
# Créer une nouvelle colonne qui concatène 'title' et 'body'
df = df.dropna()
df['text'] = df['Title'] + ' ' + df['Body']
# Séparer les données en train et test sets 
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["Tags"], test_size=0.2, random_state=42)

In [ ]:
from collections import Counter

NombreDeTags = 100
tags = [t[1:len(t) - 1].split('><') for t in df['Tags']]
words = []
for t in tags:
     words += t

freq_totale = Counter(words)
mostcommon = np.array(freq_totale.most_common(NombreDeTags))


#removing the common words from tags
final_tags = []
for ts in tags:
    final_tags.append([word for word in ts if word in mostcommon])
final_tags = np.asarray(final_tags)
n = len(max(final_tags, key=len))
liste_2 = [x + [None]*(n-len(x)) for x in final_tags]
a = np.array(liste_2)
final_tags = pd.DataFrame(final_tags, dtype = object)
ftags = pd.DataFrame(a)

In [ ]:
y = ftags.iloc[:,0]

l_cat = list(set(y))
y_cat_num = [(l_cat.index(ftags.iloc[i,0])) for i in range(len(ftags))]

In [ ]:
print("catégories : ", l_cat)


In [ ]:
# Calcul Tsne, détermination des clusters et calcul ARI entre vrais catégorie et n° de clusters
def ARI_fct(features) :
    time1 = time.time()
    num_labels=len(l_cat)
    tsne = manifold.TSNE(n_components=2, perplexity=30, n_iter=2000, 
                                 init='random', learning_rate=200, random_state=42)
    X_tsne = tsne.fit_transform(features)
    
    # Détermination des clusters à partir des données après Tsne 
    cls = cluster.KMeans(n_clusters=num_labels, n_init=100, random_state=42)
    cls.fit(X_tsne)
    ARI = np.round(metrics.adjusted_rand_score(y_cat_num, cls.labels_),4)
    time2 = np.round(time.time() - time1,0)
    print("ARI : ", ARI, "time : ", time2)
    
    return ARI, X_tsne, cls.labels_

# visualisation du Tsne selon les vraies catégories et selon les clusters
def TSNE_visu_fct(X_tsne, y_cat_num, labels, ARI) :
    fig = plt.figure(figsize=(15,6))
    
    ax = fig.add_subplot(121)
    scatter = ax.scatter(X_tsne[:,0],X_tsne[:,1], c=y_cat_num, cmap='Set1')
    ax.legend(handles=scatter.legend_elements()[0], labels=l_cat, loc="best", title="Categorie")
    plt.title('Représentation des questions par thèmes réelles')
    
    ax = fig.add_subplot(122)
    scatter = ax.scatter(X_tsne[:,0],X_tsne[:,1], c=labels, cmap='Set1')
    ax.legend(handles=scatter.legend_elements()[0], labels=set(labels), loc="best", title="Clusters")
    plt.title('Représentation des questions par clusters')
    
    plt.show()
    print("ARI : ", ARI)

### BERT

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import os
import transformers
from transformers import *

os.environ["TF_KERAS"]='1'

In [ ]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot
    

# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='HF') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                      bert_tokenizer, max_length)
        
        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                                 "input_mask" : attention_mask, 
                                 "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
             
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))
    
    features_bert = np.array(last_hidden_states_tot).mean(axis=1)
    
    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
     
    return features_bert, last_hidden_states_tot
 

In [ ]:
max_length = 64
batch_size = 8
model_type = 'bert-base-uncased'
model = TFAutoModel.from_pretrained(model_type)
sentences = df['text'].to_list()

In [ ]:
# Création des features

#features_bert, last_hidden_states_tot = feature_BERT_fct(model, model_type, sentences, 
#                                                         max_length, batch_size, mode='HF')

In [ ]:
#ARI, X_tsne, labels = ARI_fct(features_bert)

In [ ]:
#TSNE_visu_fct(X_tsne, y_cat_num, labels, ARI)

### word2vec

In [ ]:
tf.keras.backend.clear_session () 

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize


In [ ]:
def tokenizer_fct(sentence) :
    # print(sentence)
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

In [ ]:
w2v_size=100
w2v_window=5
w2v_min_count=1
w2v_epochs=100
maxlen = 24 # adapt to length of sentences
sentences = df['text'].to_list()
sentences = [gensim.utils.simple_preprocess(text) for text in sentences]

# Création et entraînement du modèle Word2Vec

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                                workers=multiprocessing.cpu_count())

w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

In [ ]:
# Préparation des sentences (tokenization)

print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 
                                                   
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Matrice d'embedding

La matrice d'embedding est une matrice qui contient les vecteurs représentant les mots d'un vocabulaire. Chaque ligne de la matrice correspond à un mot et chaque colonne correspond à une dimension du vecteur. La matrice d'embedding permet de transformer un mot en un vecteur de nombres réels qui capture son sens, son contexte et sa similarité avec d'autres mots. La matrice d'embedding peut être apprise à partir de données textuelles à l'aide de méthodes comme Word2Vec ou BERT.

In [ ]:
# Création de la matrice d'embedding

print("Create Embedding matrix ...")
w2v_size = 100
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0
    
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Modèle d'embedding

In [ ]:
# Création du modèle

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()

In [ ]:
embeddings = embed_model.predict(x_sentences)
embeddings.shape

In [ ]:
from sklearn import manifold, decomposition

In [ ]:
from sklearn import cluster, metrics

In [ ]:
ARI, X_tsne, labels = ARI_fct(embeddings)

In [ ]:
TSNE_visu_fct(X_tsne, y_cat_num, labels, ARI)

In [ ]:
tf.keras.backend.clear_session () 

 ### USE (Universal Sentence Encoder) 
 
 est un modèle qui encode les phrases dans un espace vectoriel de basse dimension, en utilisant un réseau neuronal profond.

In [ ]:
#pip install tensorflow-hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

import transformers
from transformers import *

import os

os.environ["TF_KERAS"]='1'

In [ ]:
# Charger le modèle USE

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))
    return features

In [ ]:
batch_size = 31
sentences = df['text'].to_list()
features_USE = feature_USE_fct(sentences, batch_size)
len(features_USE)

In [ ]:
features_USE

In [ ]:
ARI, X_tsne, labels = ARI_fct(features_USE)

In [ ]:
X_tsne.shape

In [ ]:
features_USE.shape

In [ ]:
TSNE_visu_fct(X_tsne, y_cat_num, labels, ARI)

#### Choix du modèle 

In [ ]:
# Sauvegarder le modèle
embed_model.save("embed_model.h5")

In [ ]:
# Charger le modèle
loaded_model = tf.keras.models.load_model("embed_model.h5")

In [ ]:
mlflow.keras.save_model(model, "models/model.pkl")

In [ ]:
model = mlflow.pyfunc.load_model("models/model.pkl")
